In [ ]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats
import plotly.graph_objects as go

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_eurchf = pd.read_csv('/content/EUR-CHF Exchange Rate.csv')
df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [ ]:
df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_eurchf['Date'] = pd.to_datetime(df_eurchf.iloc[:, 0], infer_datetime_format=True)

In [ ]:
df_trends.iloc[:,1:] = df_trends.iloc[:,1:].diff()
df_trends

,Date,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
0,2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-02,0.0,16.0,9.0,2.0,-1.0,10.0,-27.0,17.0,31.0,1.0,-2.0,0.0,1.0,-4.0,-1.0,-2.0,-6.0,2.0,2.0,0.0,-1.0,-6.0,7.0,20.0,0.0,0.0,1.0,19.0,0.0,8.0,0.0,-58.0,-1.0,3.0,-24.0,0.0,-3.0,10.0,0.0,...,-2.0,-13.0,-34.0,-13.0,18.0,19.0,6.0,0.0,-5.0,14.0,-5.0,25.0,11.0,2.0,5.0,34.0,15.0,-11.0,0.0,24.0,-13.0,0.0,-8.0,6.0,6.0,0.0,0.0,5.0,-3.0,35.0,10.0,0.0,-4.0,8.0,0.0,8.0,0.0,-16.0,6.0,10.0
2,2004-01-03,0.0,0.0,-10.0,-3.0,0.0,-6.0,0.0,-19.0,-31.0,-1.0,-3.0,0.0,-1.0,-1.0,0.0,-3.0,4.0,-2.0,0.0,0.0,0.0,5.0,-4.0,-16.0,0.0,10.0,-1.0,-6.0,0.0,-6.0,0.0,2.0,10.0,-7.0,30.0,0.0,0.0,-6.0,0.0,...,6.0,8.0,4.0,-1.0,-21.0,2.0,1.0,0.0,-4.0,-16.0,25.0,12.0,-21.0,-1.0,-6.0,-17.0,-3.0,0.0,0.0,-16.0,2.0,0.0,-8.0,-2.0,0.0,0.0,17.0,-5.0,8.0,-25.0,-2.0,0.0,8.0,7.0,0.0,-14.0,0.0,13.0,7.0,32.0
3,2004-01-04,19.0,-12.0,2.0,7.0,1.0,3.0,22.0,12.0,0.0,1.0,0.0,26.0,1.0,2.0,0.0,11.0,-5.0,1.0,1.0,2.0,0.0,0.0,-3.0,-3.0,0.0,-19.0,1.0,-8.0,0.0,2.0,0.0,7.0,-5.0,5.0,-38.0,0.0,0.0,-1.0,1.0,...,-5.0,-6.0,13.0,2.0,8.0,-16.0,3.0,14.0,5.0,7.0,3.0,-36.0,-25.0,2.0,1.0,23.0,8.0,5.0,10.0,-6.0,8.0,27.0,0.0,-5.0,7.0,0.0,-17.0,4.0,-2.0,1.0,5.0,0.0,-14.0,-9.0,0.0,7.0,0.0,18.0,5.0,-39.0
4,2004-01-05,-19.0,-7.0,-4.0,-8.0,-3.0,-5.0,-22.0,18.0,0.0,-3.0,-50.0,-26.0,-1.0,-2.0,1.0,-14.0,-16.0,-2.0,0.0,-2.0,-2.0,9.0,1.0,13.0,0.0,-3.0,9.0,2.0,0.0,-4.0,0.0,-15.0,5.0,-7.0,0.0,2.0,-1.0,-6.0,-2.0,...,-4.0,4.0,2.0,-6.0,1.0,17.0,25.0,-14.0,1.0,28.0,51.0,5.0,-16.0,4.0,5.0,-17.0,-20.0,9.0,49.0,4.0,-37.0,-27.0,0.0,11.0,11.0,0.0,0.0,9.0,35.0,13.0,11.0,0.0,7.0,38.0,0.0,-16.0,0.0,-38.0,-3.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465,2021-09-13,-1.0,0.0,4.0,0.0,6.0,32.0,17.0,8.0,3.0,1.0,-2.0,1.0,1.0,-1.0,0.0,0.0,1.0,13.0,2.0,0.0,1.0,2.0,7.0,35.0,0.0,4.0,-1.0,7.0,2.0,-1.0,-1.0,3.0,2.0,10.0,0.0,0.0,3.0,5.0,0.0,...,5.0,0.0,-1.0,-6.0,21.0,12.0,2.0,0.0,-7.0,3.0,5.0,9.0,1.0,2.0,8.0,8.0,2.0,2.0,5.0,15.0,17.0,2.0,-1.0,2.0,2.0,2.0,1.0,2.0,-1.0,10.0,1.0,0.0,6.0,1.0,0.0,2.0,0.0,-8.0,22.0,-2.0
6466,2021-09-14,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-2.0,-1.0,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-4.0,1.0,0.0,1.0,0.0,0.0,1.0,-1.0,0.0,6.0,1.0,1.0,0.0,2.0,2.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,...,-1.0,-1.0,1.0,0.0,30.0,5.0,1.0,2.0,-3.0,2.0,-1.0,-7.0,1.0,0.0,3.0,-1.0,-1.0,0.0,0.0,1.0,-2.0,1.0,1.0,-1.0,2.0,2.0,0.0,1.0,0.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,-1.0
6467,2021-09-15,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,-7.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,3.0,1.0,-1.0,0.0,-12.0,0.0,0.0,-1.0,1.0,-1.0,4.0,11.0,-1.0,0.0,0.0,7.0,4.0,1.0,1.0,3.0,-2.0,0.0,-1.0,2.0,7.0,1.0,2.0,0.0,0.0,-4.0,0.0,0.0,0.0,-7.0,-1.0,0.0,0.0,0.0,2.0,3.0
6468,2021-09-16,0.0,0.0,0.0,-1.0,-2.0,-5.0,-4.0,-1.0,-1.0,-

In [ ]:
ts_df = df_eurchf.merge(df_trends, how = 'inner', on = ['Date'])

In [ ]:
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date'], axis = 1)
ts_df = ts_df.iloc[1:,:]
ts_df

,value,log_difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-05,1.5630,0.002498,-19.0,-7.0,-4.0,-8.0,-3.0,-5.0,-22.0,18.0,0.0,-3.0,-50.0,-26.0,-1.0,-2.0,1.0,-14.0,-16.0,-2.0,0.0,-2.0,-2.0,9.0,1.0,13.0,0.0,-3.0,9.0,2.0,0.0,-4.0,0.0,-15.0,5.0,-7.0,0.0,2.0,-1.0,-6.0,...,-4.0,4.0,2.0,-6.0,1.0,17.0,25.0,-14.0,1.0,28.0,51.0,5.0,-16.0,4.0,5.0,-17.0,-20.0,9.0,49.0,4.0,-37.0,-27.0,0.0,11.0,11.0,0.0,0.0,9.0,35.0,13.0,11.0,0.0,7.0,38.0,0.0,-16.0,0.0,-38.0,-3.0,-6.0
2004-01-06,1.5675,0.002875,0.0,34.0,5.0,2.0,2.0,21.0,0.0,-20.0,30.0,5.0,77.0,0.0,0.0,9.0,0.0,7.0,25.0,2.0,3.0,0.0,3.0,-6.0,8.0,46.0,0.0,27.0,-8.0,44.0,0.0,3.0,0.0,41.0,-7.0,16.0,0.0,0.0,8.0,10.0,...,14.0,-5.0,4.0,16.0,11.0,-11.0,-14.0,12.0,20.0,-18.0,-69.0,22.0,13.0,-2.0,2.0,36.0,10.0,-9.0,-42.0,50.0,48.0,15.0,11.0,2.0,8.0,22.0,39.0,5.0,2.0,1.0,-28.0,15.0,46.0,-4.0,74.0,75.0,0.0,6.0,32.0,30.0
2004-01-07,1.5674,-0.000064,0.0,-13.0,-3.0,1.0,2.0,-3.0,0.0,-10.0,2.0,-2.0,-21.0,0.0,0.0,-1.0,1.0,7.0,-9.0,-1.0,-2.0,1.0,0.0,6.0,14.0,-16.0,0.0,1.0,1.0,-23.0,0.0,3.0,0.0,-32.0,3.0,-10.0,8.0,0.0,-4.0,-2.0,...,-2.0,2.0,-10.0,-9.0,-7.0,29.0,18.0,-3.0,2.0,-2.0,18.0,-2.0,9.0,0.0,2.0,-25.0,17.0,-1.0,2.0,-24.0,-33.0,14.0,-11.0,-9.0,1.0,-12.0,19.0,-2.0,-24.0,-8.0,6.0,-4.0,-22.0,-10.0,-34.0,-50.0,0.0,2.0,3.0,-25.0
2004-01-08,1.5653,-0.001341,0.0,7.0,5.0,1.0,-2.0,16.0,0.0,6.0,-21.0,-1.0,3.0,8.0,0.0,-5.0,0.0,3.0,4.0,3.0,-3.0,0.0,-1.0,-7.0,-8.0,-8.0,2.0,-5.0,-2.0,0.0,0.0,-2.0,0.0,-5.0,0.0,12.0,15.0,0.0,2.0,3.0,...,2.0,-4.0,5.0,3.0,5.0,-8.0,-11.0,-9.0,8.0,6.0,-8.0,9.0,18.0,-2.0,-3.0,16.0,3.0,2.0,25.0,5.0,18.0,-11.0,0.0,-5.0,-1.0,-10.0,-20.0,-3.0,-13.0,5.0,-5.0,-11.0,21.0,18.0,-40.0,26.0,0.0,-1.0,-7.0,-3.0
2004-01-09,1.5668,0.000958,0.0,8.0,-1.0,1.0,0.0,-17.0,0.0,-4.0,0.0,1.0,-6.0,-8.0,0.0,-1.0,-1.0,-7.0,-1.0,-1.0,1.0,0.0,1.0,-3.0,4.0,7.0,2.0,22.0,-1.0,-10.0,50.0,3.0,1.0,3.0,1.0,-1.0,-8.0,2.0,-2.0,-3.0,...,-1.0,3.0,39.0,0.0,-9.0,-13.0,10.0,0.0,-15.0,10.0,25.0,-4.0,-3.0,-2.0,0.0,-8.0,-30.0,1.0,-24.0,-10.0,1.0,-6.0,9.0,0.0,-14.0,0.0,-28.0,-2.0,19.0,-12.0,7.0,0.0,-3.0,-13.0,0.0,-24.0,0.0,-4.0,-13.0,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,1.0838,-0.005245,-1.0,-3.0,0.0,-2.0,-1.0,0.0,8.0,-2.0,0.0,0.0,0.0,-1.0,0.0,-2.0,0.0,-1.0,-3.0,3.0,-3.0,0.0,0.0,1.0,2.0,-4.0,2.0,0.0,-2.0,-4.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,-1.0,1.0,...,-3.0,-1.0,3.0,-1.0,-13.0,-4.0,2.0,1.0,1.0,-1.0,0.0,3.0,2.0,0.0,-1.0,0.0,-4.0,-1.0,1.0,-2.0,-7.0,-3.0,0.0,-1.0,0.0,5.0,-1.0,0.0,0.0,-1.0,-1.0,1.0,0.0,17.0,1.0,-1.0,0.0,1.0,-4.0,-1.0
2021-09-10,1.0842,0.000369,0.0,-3.0,-1.0,-1.0,-1.0,-4.0,-4.0,-6.0,-2.0,-1.0,-1.0,-1.0,-1.0,-2.0,0.0,0.0,-1.0,-4.0,-2.0,-1.0,0.0,-2.0,-1.0,-6.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,-1.0,-3.0,-1.0,0.0,0.0,-2.0,...,-2.0,0.0,0.0,1.0,7.0,-4.0,-3.0,1.0,2.0,-2.0,0.0,-2.0,-2.0,-1.0,-2.0,-3.0,1.0,-1.0,-2.0,-3.0,-4.0,-1.0,0.0,-2.0,-2.0,-3.0,0.0,-1.0,-3.0,-4.0,-1.0,-1.0,-4.0,-10.0,-2.0,0.0,0.0,2.0,-6.0,-1.0
2021-09-11,1.0846,0.000369,1.0,-3.0,-3.0,0.0,-3.0,-29.0,-5.0,-3.0,-2.0,-1.0,0.0,1.0,-1.0,0.0,0.0,-1.0,0.0,-11.0,-5.0,0.0

In [ ]:
#Convert EUR/CHF to CHF/EUR Returns
ts_df['log_difference'] = ts_df['log_difference'] * -1

In [ ]:
t_test = stats.ttest_ind(ts_df['log_difference'], ts_df['swiss franc'])
t_test

Ttest_indResult(statistic=-1.159423553151061, pvalue=0.2463128687306873)

In [ ]:
trend_names = list(ts_df.columns)
statistic = []
p_value = []

In [ ]:
for column in ts_df:
  t_test = stats.ttest_ind(ts_df['log_difference'], ts_df[column])
  statistic.append(t_test[0])
  p_value.append(t_test[1])

In [ ]:
t_tests = pd.DataFrame(columns=['Trend Name','Statistic','p-value'])
t_tests['p-value'] = p_value
t_tests['Statistic'] = statistic
t_tests['Trend Name'] = trend_names

In [ ]:
t_tests.sort_values(by=['p-value'], ascending=False)
#t_tests

,Trend Name,Statistic,p-value
1,log_difference,0.000000,1.000000e+00
72,water,0.009831,9.921563e-01
113,pandemic,-0.137331,8.907724e-01
46,conflict,-0.200270,8.412740e-01
138,diversification,0.221516,8.246954e-01
...,...,...,...
25,finance,-18.781463,2.706290e-77
102,banking,-19.935788,1.183583e-86
50,bank,-23.927587,6.786863e-123
86,credit,-24.842065,5.072934e-132


In [ ]:
with open('CHFEUR T-Test.tex', 'w') as tf:
     tf.write(t_tests.to_latex())

#Rolling T-Test

In [ ]:
index_dates = ts_df.iloc[100:len(ts_df)-100].index

In [ ]:
rolling_statistic = []
rolling_p_value = []

for i in range(100, len(ts_df), 1):
  sub = ts_df.iloc[i-99:i,:]
  sub_t_test = stats.ttest_ind(sub['log_difference'], sub['swiss franc'])
  rolling_statistic.append(sub_t_test[0])
  rolling_p_value.append(sub_t_test[1])

fig1 = go.Figure([go.Scatter(name = 'CHF/EUR', x=index_dates, y=rolling_p_value)])

fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="p-value",
    width=500,
    height=500,
    font = dict(
        size=25
    )
)

fig1.add_shape(
        type='line',
        x0=index_dates.min(),
        y0=0.05,
        x1=index_dates.max(),
        y1=0.05,
        line=dict(
            color='Red',
        )
)

fig1.show()

In [ ]:
rolling_statistic = []
rolling_p_value = []

for i in range(100, len(ts_df), 1):
  sub = ts_df.iloc[i-99:i,:]
  sub_t_test = stats.ttest_ind(sub['log_difference'], sub['crisis'])
  rolling_statistic.append(sub_t_test[0])
  rolling_p_value.append(sub_t_test[1])

fig1 = go.Figure([go.Scatter(name = 'CHF/EUR', x=index_dates, y=rolling_p_value)])

fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="p-value",
    width=500,
    height=500,
    font = dict(
        size=25
    )
)

fig1.add_shape(
        type='line',
        x0=index_dates.min(),
        y0=0.05,
        x1=index_dates.max(),
        y1=0.05,
        line=dict(
            color='Red',
        )
)

fig1.show()

In [ ]:
rolling_statistic = []
rolling_p_value = []

for i in range(100, len(ts_df), 1):
  sub = ts_df.iloc[i-99:i,:]
  sub_t_test = stats.ttest_ind(sub['log_difference'], sub['financial crisis'])
  rolling_statistic.append(sub_t_test[0])
  rolling_p_value.append(sub_t_test[1])

fig1 = go.Figure([go.Scatter(name = 'CHF/EUR', x=index_dates, y=rolling_p_value)])

fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="p-value",
    width=500,
    height=500,
    font = dict(
        size=25
    )
)

fig1.add_shape(
        type='line',
        x0=index_dates.min(),
        y0=0.05,
        x1=index_dates.max(),
        y1=0.05,
        line=dict(
            color='Red',
        )
)

fig1.show()

In [ ]:
rolling_statistic = []
rolling_p_value = []

for i in range(100, len(ts_df), 1):
  sub = ts_df.iloc[i-99:i,:]
  sub_t_test = stats.ttest_ind(sub['log_difference'], sub['leverage'])
  rolling_statistic.append(sub_t_test[0])
  rolling_p_value.append(sub_t_test[1])

fig1 = go.Figure([go.Scatter(name = 'CHF/EUR', x=index_dates, y=rolling_p_value)])

fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="p-value",
    width=500,
    height=500,
    font = dict(
        size=25
    )
)

fig1.add_shape(
        type='line',
        x0=index_dates.min(),
        y0=0.05,
        x1=index_dates.max(),
        y1=0.05,
        line=dict(
            color='Red',
        )
)

fig1.show()

In [ ]:
rolling_statistic = []
rolling_p_value = []

for i in range(100, len(ts_df), 1):
  sub = ts_df.iloc[i-99:i,:]
  sub_t_test = stats.ttest_ind(sub['log_difference'], sub['uncertainty'])
  rolling_statistic.append(sub_t_test[0])
  rolling_p_value.append(sub_t_test[1])

fig1 = go.Figure([go.Scatter(name = 'CHF/EUR', x=index_dates, y=rolling_p_value)])

fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="p-value",
    width=500,
    height=500,
    font = dict(
        size=25
    )
)

fig1.add_shape(
        type='line',
        x0=index_dates.min(),
        y0=0.05,
        x1=index_dates.max(),
        y1=0.05,
        line=dict(
            color='Red',
        )
)

fig1.show()